<div style="text-align: right"> <b>Release Date:</b> 20.11.2023 </div>

<div style="text-align: right"> v.2.0.0 </div>

---

<h3><center> FeVAcS: FEniCS Visualizing Acoustic Scattering </center></h3>
<h3><center> Notebook: FEM </center></h3>
<h4><center>Mete Öğüç<sup>1</sup>, Ali Fethi Okyar<sup>2</sup>, Tahsin Khajah<sup>3</sup></center></h4>
<t4><center> <sup>1</sup>İstanbul Atlas University, Biomedical Eng. Dept. &amp; Metzen Akustik</center></t4>
<t4><center> <sup>2</sup>Yeditepe University, Mech. Eng. Dept. </center></t4>
<t4><center> <sup>3</sup>The University of Texas at Tyler, Mech. Eng. Dept. </center></t4>

---

This notebook contains the necessary functions to generate mesh and solve the finite element model.

#### Mesh the geometry

In [ ]:
def Mesh_Cylinder(xmin, xmax, ymin, ymax, xcyl_1, ycyl_1, r_1, n_lambda, k):

    # Create the solution domain
    domain_rectangle = Rectangle(Point(xmin, ymin), Point(xmax, ymax))
    
    n_x = int(n_lambda*(xmax-xcyl_1)*k/(2*np.pi))
    n_tangential = int(n_x*2*np.pi)
    
    # Single cylinder shaped obstacle
    cylinder_1  = Circle(Point(xcyl_1, ycyl_1), r_1, n_tangential)
    
    domain = domain_rectangle - cylinder_1
    mesh = generate_mesh(domain, int(n_x*2))
    
    return mesh

In [ ]:
mesh = Mesh_Cylinder(xmin, xmax, ymin, ymax, xcyl_1, ycyl_1, r_1, n_lambda, k)

#### Extract DOF coordinates of the mesh

In [ ]:
F = FunctionSpace(mesh, 'P', p)

n = F.dim()
d = mesh.geometry().dim()

F_dof_coordinates = F.tabulate_dof_coordinates()
F_dof_coordinates.resize((n, d))
F_dof_x = F_dof_coordinates[:, 0]
F_dof_y = F_dof_coordinates[:, 1]

#### Generate plane wave

In [ ]:
def Plane_Wave(k, x, dw):
    
    u_inc = np.cos(k*(x-dw)) + 1j*np.sin(k*(x-dw))
    
    return u_inc

#### Finite Element Model

In [ ]:
def FEM(mesh, k, xmin, xmax, ymin, ymax, xcyl_1, ycyl_1, r_1, p, q, tol):
             
    # Define boundary subdomains
    
    boundary_markers = MeshFunction("size_t",mesh, mesh.topology().dim()-1)
    
    class BoundaryL(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], xmin, tol)
    
    class BoundaryR(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], xmax, tol)
    
    class BoundaryS(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and sqrt(pow(xcyl_1 - x[0],2) + pow(ycyl_1 - x[1],2)) <= r_1 + tol

    # Sub domain for Periodic boundary condition
    class PeriodicBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return bool( ymin + tol > x[1] > ymin - tol and on_boundary)
        def map(self, x, y):
            y[0] = x[0]
            y[1] = x[1] - (ymax-ymin)
    
    bc_L = BoundaryL()
    bc_R = BoundaryR()
    bc_S = BoundaryS()

    boundary_markers.set_all(0)
    bc_L.mark(boundary_markers, 1)
    bc_R.mark(boundary_markers, 2)
    bc_S.mark(boundary_markers, 3)
    
    Er = FiniteElement('P', triangle, p)
    Ei = FiniteElement('P', triangle, p)
    Ec = Er * Ei
    V = FunctionSpace(mesh, Ec, constrained_domain=PeriodicBoundary())
    print("DOF of Functions-space V:", round(V.dim()/2))
    
    C_1 = Expression('k*cos(atan2(ycyl_1 + x[1],x[0] - xcyl_1))*cos(k*(x[0] - xcyl_1))',degree=q, k=k, xcyl_1 = xcyl_1, ycyl_1 = ycyl_1)
    C_2 = Expression('k*cos(atan2(ycyl_1 + x[1],x[0] - xcyl_1))*sin(k*(x[0] - xcyl_1))',degree=q, k=k, xcyl_1 = xcyl_1, ycyl_1 = ycyl_1)
    
    # Redefine boundary integration measure
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
    
    # define variational problem
    (u_r, u_i) = TrialFunction(V)
    (v_r, v_i) = TestFunction(V)

    a_r = \
    + inner(nabla_grad(u_r), nabla_grad(v_r))*dx\
    - inner(nabla_grad(u_i), nabla_grad(v_i))*dx\
    - pow(k,2)*( inner(u_r,v_r) - inner(u_i,v_i))*dx\
    + k*(inner(u_r,v_i) + inner(u_i,v_r))*ds(1)\
    + k*(inner(u_r,v_i) + inner(u_i,v_r))*ds(2)

    a_i = \
    + inner(nabla_grad(u_r), nabla_grad(v_i))*dx\
    + inner(nabla_grad(u_i), nabla_grad(v_r))*dx\
    - pow(k,2)*( inner(u_r,v_i) + inner(u_i,v_r))*dx\
    - k*(inner(u_r,v_r) - inner(u_i,v_i))*ds(1)\
    - k*(inner(u_r,v_r) - inner(u_i,v_i))*ds(2)
    
    L_r = -(C_2*v_r + C_1*v_i)*ds(3)
    L_i =  (C_1*v_r - C_2*v_i)*ds(3)
    
    a = a_r + a_i
    L = L_r + L_i

    # compute solution
    u = Function(V)
    solve(a == L, u)

    return u

In [ ]:
u_s = FEM(mesh, k, xmin, xmax, ymin, ymax, xcyl_1, ycyl_1, r_1, p, q, tol)
(u_s_r, u_s_i) = split(u_s)
u_s_m = sqrt(pow(u_s_r,2) + pow(u_s_i,2))

In [ ]:
def plot_results(fsize, u_p_r, u_m, run_no):

    plt.subplots(1, 2, figsize=(8,4))

    plt.subplot(1, 2, 1)
    fig = plot(u_p_r, cmap='jet')

    plt.title('Plane wave', pad=15, fontsize=fsize)
    plt.xlabel(r'$x$', fontsize=fsize)
    plt.ylabel(r'$y$', fontsize=fsize)

    colorbar = plt.colorbar(fig, fraction=0.046, pad=0.04);
    colorbar.ax.get_yaxis().labelpad = 20
    colorbar.ax.set_ylabel('$\Re \{ u_\mathregular{inc} \}$', rotation=270, fontsize=fsize)

    tick_locator = ticker.MaxNLocator(nbins=2)
    colorbar.locator = tick_locator
    colorbar.update_ticks()

    plt.subplot(1, 2, 2)
    color_min = 0
    color_max = 2.5
    fig = plot(u_m, cmap='jet', mode='color', vmin=color_min, vmax=color_max)
    plt.title('Total Wave', pad=15, fontsize=fsize)
    plt.xlabel(r'$x$', fontsize=fsize)
    plt.ylabel(r'$y$', fontsize=fsize)

    plt.locator_params(axis='y', nbins=2)
    plt.locator_params(axis='x', nbins=2)
    
    colorbar = plt.colorbar(fig, fraction=0.046, pad=0.04, ticks=[color_min, (color_min+color_max)/2, color_max]);
    colorbar.ax.get_yaxis().labelpad = 20
    colorbar.ax.set_ylabel('$|u|$', rotation=270, fontsize=fsize)

    plt.tight_layout()

    plt.savefig("Images/Graph_" + str(run_no) +".png", format="PNG", bbox_inches='tight')
    
    plt.close()

---
**Release Notes**

**v.1.0.0**<br>
02.06.2023<br>
Draft version.<br>

**v.2.0.0**<br>
20.11.2023<br>
Release version.<br>